In [2]:
import pandas as pd
import sqlite3
import numpy as np
import random
import datetime
#import matplotlib as plt
from matplotlib import pyplot as plt

In [4]:
db_yahoo = '/home/ian/Data/yahoo.db'
database_yahoo = sqlite3.connect(db_yahoo)
yahoo_string = '''SELECT * FROM divs;'''

df_yahoo = pd.read_sql_query(yahoo_string, database_yahoo)
                       
db_advfn = '/home/ian/Data/advfn.db'
database_advfn = sqlite3.connect(db_advfn)

advfn_string = '''SELECT company_ticker, report_date, `Basic EPS (Continuing)` as eps FROM quarterly_reports;'''

df_advfn = pd.read_sql_query(advfn_string, database_advfn)

db_tmx = '/home/ian/Data/tmx.db'
database_tmx = sqlite3.connect(db_tmx)

tmx_string = '''SELECT * FROM tmx_earnings;'''

df_tmx = pd.read_sql_query(tmx_string, database_tmx)

price_string = '''SELECT symbol, Date, Close FROM tsx_prices'''
df_price = pd.read_sql_query(price_string, database_yahoo)


In [5]:
df_advfn['symbol'] = df_advfn['company_ticker'].apply(lambda x: x.replace('.', '-'))

In [6]:
df_advfn['symbol'].unique()

array(['AAV', 'ABX', 'AC', 'ACB', 'ACO-X', 'AD', 'AEM', 'AGI', 'AIF',
       'ALA', 'AP-UN', 'APH', 'AQN', 'ARE', 'ARX', 'ASR', 'ATA', 'ATD-B',
       'AX-UN', 'BAD', 'BAM-A', 'BB', 'BBD-B', 'BBU-UN', 'BCB', 'BCE',
       'BEI-UN', 'BEP-UN', 'BHC', 'BIP-UN', 'BIR', 'BLX', 'BMO', 'BNS',
       'BPY-UN', 'BTE', 'BTO', 'BYD-UN', 'CAE', 'CAR-UN', 'CAS', 'CCA',
       'CCL-B', 'CCO', 'CEU', 'CFP', 'CG', 'CGX', 'CHE-UN', 'CHP-UN',
       'CHR', 'CIGI', 'CIX', 'CLS', 'CM', 'CMG', 'CNQ', 'CNR', 'CP',
       'CPG', 'CPX', 'CRR-UN', 'CSH-UN', 'CSU', 'CTC-A', 'CU', 'CUF-UN',
       'CVE', 'CWB', 'D-UN', 'DGC', 'DII-B', 'DOL', 'DOO', 'DRG-UN',
       'DSG', 'ECA', 'ECN', 'EDV', 'EFN', 'EFX', 'EIF', 'ELD', 'EMA',
       'EMP-A', 'ENB', 'ENGH', 'ERF', 'ESI', 'EXE', 'FCR', 'FFH', 'FM',
       'FNV', 'FR', 'FRU', 'FSV', 'FTS', 'FTT', 'FVI', 'G', 'GC', 'GEI',
       'GIB-A', 'GIL', 'GOOS', 'GRT-UN', 'GTE', 'GUD', 'GUY', 'GWO', 'H',
       'HBC', 'HBM', 'HCG', 'HR-UN', 'HSE', 'IAG', 'IFC', 'IFP', 'IGM',

In [7]:
df_yahoo['parsed_date'] = df_yahoo['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_yahoo.head(2)

,symbol,Date,Dividends,parsed_date
0,ABX,2018-12-27,0.07,2018-12-27
1,ABX,2018-11-29,0.03,2018-11-29


In [8]:
df_advfn['parsed_date'] = df_advfn['report_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m'))
df_advfn.head(2)

,company_ticker,report_date,eps,symbol,parsed_date
0,AAV,2017/06,0.10,AAV,2017-06-01
1,AAV,2017/09,0.07,AAV,2017-09-01


In [11]:
df_price['parsed_date'] = df_price['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_price.head(5)

,symbol,Date,Close,parsed_date
0,AAV,2009-10-29,7.130000,2009-10-29
1,AAV,2009-10-30,6.760000,2009-10-30
2,AAV,2009-11-02,6.250000,2009-11-02
3,AAV,2009-11-03,6.740000,2009-11-03
4,AAV,2009-11-04,6.730000,2009-11-04


In [13]:
df_tmx['parsed_date'] = df_tmx['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_tmx.head(5)

,symbol,date,eps,consensus_eps,surprise,parsed_date
0,RCI.B,2019-04-18,0.78,0.94,-17.02,2019-04-18
1,RCI.B,2019-01-24,1.13,1.08,4.63,2019-01-24
2,RCI.B,2018-10-19,1.21,1.16,4.31,2018-10-19
3,RCI.B,2018-07-19,1.07,1.02,4.90,2018-07-19
4,RCI.B,2018-04-19,0.90,0.74,21.62,2018-04-19


In [14]:
df_merge = pd.concat([df_tmx[['symbol', 'parsed_date', 'eps']], df_yahoo[['symbol', 'parsed_date', 'Dividends']], df_price[['symbol', 'parsed_date', 'Close']] ])
df_merge.head(5)

/home/ian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Close,Dividends,eps,parsed_date,symbol
0,NaN,NaN,0.78,2019-04-18,RCI.B
1,NaN,NaN,1.13,2019-01-24,RCI.B
2,NaN,NaN,1.21,2018-10-19,RCI.B
3,NaN,NaN,1.07,2018-07-19,RCI.B
4,NaN,NaN,0.90,2018-04-19,RCI.B


In [15]:
df_merge.sort_values(by = ['symbol','parsed_date','Dividends'], ascending = [True, False, False], inplace = True )


,symbol,parsed_date,Close,eps,Dividends
61,AAV,2019-05-02,NaN,0.00,NaN
62,AAV,2019-02-28,NaN,0.14,NaN
2333,AAV,2019-02-15,2.330000,NaN,NaN
2332,AAV,2019-02-14,2.260000,NaN,NaN
2331,AAV,2019-02-13,2.330000,NaN,NaN
2330,AAV,2019-02-12,2.170000,NaN,NaN
2329,AAV,2019-02-11,1.920000,NaN,NaN
2328,AAV,2019-02-08,1.900000,NaN,NaN
2327,AAV,2019-02-07,1.980000,NaN,NaN
2326,AAV,2019-02-06,2.090000,NaN,NaN


In [17]:
df_bns = df_merge[['symbol', 'parsed_date', 'Close', 'eps', 'Dividends']][df_merge['symbol'] == 'BNS'].copy()

In [22]:
df_bns.fillna(method='ffill', inplace = True)
df_bns['p-e'] = 0
df_bns['yld'] = 0
df_bns['p-e'] = df_bns['Close'] / df_bns['eps']

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''